In [1]:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluation-metrics.md

In [2]:
2+2

4

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import json
import sys
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
from elasticsearch import Elasticsearch, ConnectionError

try:
    es = Elasticsearch(['http://elasticsearch:9200'])  # Use the service name
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except ConnectionError as e:
    print(f"Connection error: {e}")


Connected to Elasticsearch


In [86]:
# from elasticsearch import Elasticsearch, NotFoundError, ConnectionError
# from tqdm import tqdm
# import json

# class ReviewIndexer:
#     def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
#         self.es = Elasticsearch([es_host])
#         self.index_name = index_name
#         self.model = model  # Expecting a SentenceTransformer model to encode text
        
#         # Check the connection upon initialization
#         self.check_connection()
        
#         self.index_settings = {
#             "settings": {
#                 "number_of_shards": 1,
#                 "number_of_replicas": 0
#             },
#             "mappings": {
#                 "properties": {
#                     "appid": {"type": "keyword"},
#                     "timestamp_query": {"type": "integer"},
#                     "title": {"type": "keyword"},
#                     "recommendationid": {"type": "keyword"},
#                     "author.steamid": {"type": "keyword"},
#                     "author.playtimeforever": {"type": "integer"},
#                     "author.playtime_last_two_weeks": {"type": "integer"},
#                     "author.playtime_at_review": {"type": "integer"},
#                     "author.last_played": {"type": "integer"},
#                     "language": {"type": "keyword"},
#                     "review": {"type": "text"},
#                     "voted_up": {"type": "boolean"},
#                     "votes_up": {"type": "integer"},
#                     "timestamp_created": {"type": "integer"},
#                     "timestamp_updated": {"type": "integer"},
#                     "title_vector": {
#                         "type": "dense_vector",
#                         "dims": 384,
#                         "index": True,
#                         "similarity": "cosine"
#                     },
#                     "review_vector": {
#                         "type": "dense_vector",
#                         "dims": 384,
#                         "index": True,
#                         "similarity": "cosine"
#                     },
#                     "title_review_vector": {
#                         "type": "dense_vector",
#                         "dims": 384,
#                         "index": True,
#                         "similarity": "cosine"
#                     },
#                 }
#             }
#         }

#         # Create index if it does not exist
#         self.create_index()

#     def check_connection(self):
#         """Check if Elasticsearch connection is established."""
#         try:
#             self.es.ping()
#             print("Connected to Elasticsearch!")
#         except ConnectionError:
#             print("Failed to connect to Elasticsearch.")

#     def create_index(self):
#         """Create the Elasticsearch index with specified settings and mappings."""
#         if not self.es.indices.exists(index=self.index_name):
#             self.es.indices.create(index=self.index_name, body=self.index_settings)
#             print(f"Index '{self.index_name}' created.")
#         else:
#             print(f"Index '{self.index_name}' already exists.")

#     def index_reviews(self, reviews):
#         """Index the provided reviews into Elasticsearch."""
#         for review in tqdm(reviews):
#             # Generate vectors for title, review, and a combined title + review
#             title_vector = self.model.encode(review["review"]["title"]) if review["review"]["title"] else [0]*384
#             review_vector = self.model.encode(review["review"]["review"]) if review["review"]["review"] else [0]*384
#             combined_text = f"{review['review']['title']} {review['review']['review']}" if review['review']['title'] and review['review']['review'] else ""
#             title_review_vector = self.model.encode(combined_text) if combined_text else [0]*384

#             # Prepare the document to be indexed
#             doc = {
#                 "appid": review["appid"],
#                 "timestamp_query": review["review"]["timestamp_query"],
#                 "title": review["review"]["title"],
#                 "recommendationid": review["review"]["recommendationid"],
#                 "author.steamid": review["review"]["author.steamid"],
#                 "author.playtimeforever": review["review"]["author.playtimeforever"],
#                 "author.playtime_last_two_weeks": review["review"]["author.playtime_last_two_weeks"],
#                 "author.playtime_at_review": review["review"]["author.playtime_at_review"],
#                 "author.last_played": review["review"]["author.last_played"],
#                 "language": review["review"]["language"],
#                 "review": review["review"]["review"],
#                 "voted_up": review["review"]["voted_up"],
#                 "votes_up": review["review"]["votes_up"],
#                 "timestamp_created": review["review"]["timestamp_created"],
#                 "timestamp_updated": review["review"]["timestamp_updated"],
#                 "title_vector": title_vector,
#                 "review_vector": review_vector,
#                 "title_review_vector": title_review_vector
#             }

#             # Index the document
#             try:
#                 self.es.index(index=self.index_name, body=doc)
#             except Exception as e:
#                 print(f"Error indexing document with appid {review['appid']}: {e}")

# # Example usage
# if __name__ == "__main__":
#     from sentence_transformers import SentenceTransformer

#     # Initialize the model
#     model_name = 'multi-qa-MiniLM-L6-cos-v1'
#     model = SentenceTransformer(model_name)

#     # Sample JSON input
#     sample_reviews = [
#         {
#             "appid": "552520",
#             "review": {
#                 "appid": "552520",
#                 "timestamp_query": 1727643597,
#                 "title": "Far Cry 5",
#                 "recommendationid": "175498502",
#                 "author.steamid": "76561198275890433",
#                 "author.playtimeforever": None,
#                 "author.playtime_last_two_weeks": 0,
#                 "author.playtime_at_review": 2601,
#                 "author.last_played": 1726001587,
#                 "language": "english",
#                 "review": "If you are a fan of yet another generic open-world ubisoft game where you capture outposts and unlock areas, this is the perfect game for you, if you are an otherwise a normal person stay away from this generic cashgrab, graphics are clearly improved from far cry 4, but that's about it - the graphics. Nothing else has changed at all and it feels like a reskin, the npcs are absolutely horrendous and buggy, the story missions are bland, atleast the villain is half-alright. The shooting is boring, no recoil, no feel and it doesn't encourage you to buy new guns since the starter one can carry you till the end of the game. Typical ubisoft generic release.",
#                 "voted_up": False,
#                 "votes_up": 1,
#                 "timestamp_created": 1726965394,
#                 "timestamp_updated": 1726965394
#             },
#             "question": "Is this game a complete and polished experience upon release?",
#             "answer": "No, it feels like a reskin and has many bugs.",
#             "section": "completeness"
#         },{
#         "appid": "2322010",
#         "review": {
#             "appid": "2322010",
#             "timestamp_query": 1727643597,
#             "title": "God of War: Ragnarok",
#             "recommendationid": "175341259",
#             "author.steamid": "76561198053458540",
#             "author.playtimeforever": None,
#             "author.playtime_last_two_weeks": 65,
#             "author.playtime_at_review": 48,
#             "author.last_played": 1727486181,
#             "language": "english",
#             "review": "This game launching without proper controller support when God of War had it is seriously a shame. Hopefully this is a bug and not some scam to sell you a shitty PlayStation controller. Fix this soon before I am outside of my refund window.\n\nEdit: I've spent another 30 minutes trying every janky ass fix you can find online and in the discussions and shockingly none of them work. Seriously, how is this a thing? And no fix yet? CLUELESS FUCKING DEVS MAN, IT'S GETTING SO TIRING!",
#             "voted_up": False,
#             "votes_up": 16,
#             "timestamp_created": 1726793968,
#             "timestamp_updated": 1726900092
#         },
#         "question": "Is the controller support functional at launch?",
#         "answer": "No, it launched without proper controller support.",
#         "section": "game mechanics"
#     },
#     {
#         "appid": "2322010",
#         "review": {
#             "appid": "2322010",
#             "timestamp_query": 1727643597,
#             "title": "God of War: Ragnarok",
#             "recommendationid": "175341259",
#             "author.steamid": "76561198053458540",
#             "author.playtimeforever": None,
#             "author.playtime_last_two_weeks": 65,
#             "author.playtime_at_review": 48,
#             "author.last_played": 1727486181,
#             "language": "english",
#             "review": "This game launching without proper controller support when God of War had it is seriously a shame. Hopefully this is a bug and not some scam to sell you a shitty PlayStation controller. Fix this soon before I am outside of my refund window.\n\nEdit: I've spent another 30 minutes trying every janky ass fix you can find online and in the discussions and shockingly none of them work. Seriously, how is this a thing? And no fix yet? CLUELESS FUCKING DEVS MAN, IT'S GETTING SO TIRING!",
#             "voted_up": False,
#             "votes_up": 16,
#             "timestamp_created": 1726793968,
#             "timestamp_updated": 1726900092
#         },
#         "question": "Have the developers acknowledged the control issues?",
#         "answer": "No acknowledgment mentioned from the developers.",
#         "section": "developers"
#     },
#         # Add more reviews as needed
#     ]

#     # Create an indexer instance
#     indexer = ReviewIndexer(model=model)
#     # Index the sample reviews
#     indexer.index_reviews(sample_reviews)


In [146]:
from elasticsearch import Elasticsearch, NotFoundError, ConnectionError
from tqdm import tqdm
import json

class ReviewIndexer:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Expecting a SentenceTransformer model to encode text
        
        # Check the connection upon initialization
        self.check_connection()
        
        self.index_settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "appid": {"type": "keyword"},
                    "timestamp_query": {"type": "integer"},
                    "title": {"type": "keyword"},
                    "recommendationid": {"type": "keyword"},
                    "author.steamid": {"type": "keyword"},
                    "author.playtimeforever": {"type": "integer"},
                    "author.playtime_last_two_weeks": {"type": "integer"},
                    "author.playtime_at_review": {"type": "integer"},
                    "author.last_played": {"type": "integer"},
                    "language": {"type": "keyword"},
                    "review": {"type": "text"},
                    "voted_up": {"type": "boolean"},
                    "votes_up": {"type": "integer"},
                    "timestamp_created": {"type": "integer"},
                    "timestamp_updated": {"type": "integer"},
                    "title_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "title_review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                }
            }
        }

        # Drop the index if it exists and create a new one
        self.drop_and_create_index()

    def check_connection(self):
        """Check if Elasticsearch connection is established."""
        try:
            self.es.ping()
            print("Connected to Elasticsearch!")
        except ConnectionError:
            print("Failed to connect to Elasticsearch.")

    def drop_and_create_index(self):
        """Delete the existing index if it exists and create a new one."""
        try:
            if self.es.indices.exists(index=self.index_name):
                self.es.indices.delete(index=self.index_name)
                print(f"Index '{self.index_name}' deleted.")
            self.es.indices.create(index=self.index_name, body=self.index_settings)
            print(f"Index '{self.index_name}' created.")
        except Exception as e:
            print(f"Error creating index: {e}")

    def index_reviews(self, reviews):
        """Index the provided reviews into Elasticsearch."""
        for review in tqdm(reviews):
            # Generate vectors for title, review, and a combined title + review
            title_vector = self.model.encode(review["review"]["title"]) if review["review"]["title"] else [0]*384
            review_vector = self.model.encode(review["review"]["review"]) if review["review"]["review"] else [0]*384
            combined_text = f"{review['review']['title']} {review['review']['review']}" if review['review']['title'] and review['review']['review'] else ""
            title_review_vector = self.model.encode(combined_text) if combined_text else [0]*384

            # Prepare the document to be indexed
            doc = {
                "appid": review["appid"],
                "timestamp_query": review["review"]["timestamp_query"],
                "title": review["review"]["title"],
                "recommendationid": review["review"]["recommendationid"],
                "author.steamid": review["review"]["author.steamid"],
                "author.playtimeforever": review["review"]["author.playtimeforever"],
                "author.playtime_last_two_weeks": review["review"]["author.playtime_last_two_weeks"],
                "author.playtime_at_review": review["review"]["author.playtime_at_review"],
                "author.last_played": review["review"]["author.last_played"],
                "language": review["review"]["language"],
                "review": review["review"]["review"],
                "voted_up": review["review"]["voted_up"],
                "votes_up": review["review"]["votes_up"],
                "timestamp_created": review["review"]["timestamp_created"],
                "timestamp_updated": review["review"]["timestamp_updated"],
                "title_vector": title_vector,
                "review_vector": review_vector,
                "title_review_vector": title_review_vector
            }

            # Index the document
            try:
                self.es.index(index=self.index_name, body=doc)
            except Exception as e:
                print(f"Error indexing document with appid {review['appid']}: {e}")

# Example usage
if __name__ == "__main__":
    from sentence_transformers import SentenceTransformer

    # Initialize the model
    model_name = 'multi-qa-MiniLM-L6-cos-v1'
    model = SentenceTransformer(model_name)

    # Sample JSON input
    sample_reviews = [
        {
            "appid": "552520",
            "review": {
                "appid": "552520",
                "timestamp_query": 1727643597,
                "title": "Far Cry 5",
                "recommendationid": "175498502",
                "author.steamid": "76561198275890433",
                "author.playtimeforever": None,
                "author.playtime_last_two_weeks": 0,
                "author.playtime_at_review": 2601,
                "author.last_played": 1726001587,
                "language": "english",
                "review": "If you are a fan of yet another generic open-world ubisoft game where you capture outposts and unlock areas, this is the perfect game for you, if you are an otherwise a normal person stay away from this generic cashgrab...",
                "voted_up": False,
                "votes_up": 1,
                "timestamp_created": 1726965394,
                "timestamp_updated": 1726965394
            },
            "question": "Is this game a complete and polished experience upon release?",
            "answer": "No, it feels like a reskin and has many bugs.",
            "section": "completeness"
        },
        {
        "appid": "2322010",
        "review": {
            "appid": "2322010",
            "timestamp_query": 1727643597,
            "title": "God of War: Ragnarok",
            "recommendationid": "175341259",
            "author.steamid": "76561198053458540",
            "author.playtimeforever": None,
            "author.playtime_last_two_weeks": 65,
            "author.playtime_at_review": 48,
            "author.last_played": 1727486181,
            "language": "english",
            "review": "This game launching without proper controller support when God of War had it is seriously a shame. Hopefully this is a bug and not some scam to sell you a shitty PlayStation controller. Fix this soon before I am outside of my refund window.\n\nEdit: I've spent another 30 minutes trying every janky ass fix you can find online and in the discussions and shockingly none of them work. Seriously, how is this a thing? And no fix yet? CLUELESS FUCKING DEVS MAN, IT'S GETTING SO TIRING!",
            "voted_up": False,
            "votes_up": 16,
            "timestamp_created": 1726793968,
            "timestamp_updated": 1726900092
        },
        "question": "Is the controller support functional at launch?",
        "answer": "No, it launched without proper controller support.",
        "section": "game mechanics"
        },
        # Add more reviews as needed
    ]

    # Create an indexer instance
    indexer = ReviewIndexer(model=model)
    # Index the sample reviews
    indexer.index_reviews(sample_reviews)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Connected to Elasticsearch!
Index 'steam-reviews' deleted.
Index 'steam-reviews' created.


100%|██████████| 2/2 [00:00<00:00, 12.36it/s]


In [147]:
# Code to check mappings in Elasticsearch
from elasticsearch import Elasticsearch

def check_index_mapping(es_host='http://elasticsearch:9200', index_name='steam-reviews'):
    es = Elasticsearch([es_host])

    try:
        # Get the index mappings
        response = es.indices.get_mapping(index=index_name)
        mapping = response[index_name]['mappings']
        
        # Check for vector fields and their dimensions
        vector_fields = ["review_vector", "title_vector", "title_review_vector"]
        
        for field in vector_fields:
            if field in mapping['properties']:
                dims = mapping['properties'][field]['dims']
                print(f"{field}: {dims} dimensions")
            else:
                print(f"{field} not found in index.")
    except Exception as e:
        print(f"Error retrieving index mapping: {e}")

# Run the check
check_index_mapping()


review_vector: 384 dimensions
title_vector: 384 dimensions
title_review_vector: 384 dimensions


In [148]:
from sentence_transformers import SentenceTransformer

def check_vector_dimensions(query, model_name='multi-qa-MiniLM-L6-cos-v1'):
    # Load the model
    model = SentenceTransformer(model_name)

    # Encode the query and get the vector
    vector = model.encode(query)

    # Check the dimensions of the vector
    vector_dimensions = len(vector)
    print(f"Generated vector has {vector_dimensions} dimensions.")

# Example Usage
query = 'Is God of War: Ragnarok a game for kids?'
check_vector_dimensions(query)

Generated vector has 384 dimensions.


In [149]:
from elasticsearch import Elasticsearch

def verify_appid_exists(es_host='http://elasticsearch:9200', index_name='steam-reviews', appid='2322010'):
    es = Elasticsearch([es_host])

    try:
        # Define a query to check if the appid exists
        search_query = {
            "query": {
                "term": {
                    "appid": appid  # Filter by appid
                }
            }
        }

        # Perform the search
        es_results = es.search(index=index_name, body=search_query)

        if es_results['hits']['total']['value'] > 0:
            print(f"AppID '{appid}' exists in the index.")
            for hit in es_results['hits']['hits']:
                print(hit['_source'])
        else:
            print(f"AppID '{appid}' not found in the index.")

    except Exception as e:
        print(f"Error checking for AppID: {e}")

# Run the check for appid 2322010 (God of War: Ragnarok)
# verify_appid_exists(appid="552520")
verify_appid_exists(appid="2322010")

AppID '2322010' exists in the index.
{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175341259', 'author.steamid': '76561198053458540', 'author.playtimeforever': None, 'author.playtime_last_two_weeks': 65, 'author.playtime_at_review': 48, 'author.last_played': 1727486181, 'language': 'english', 'review': "This game launching without proper controller support when God of War had it is seriously a shame. Hopefully this is a bug and not some scam to sell you a shitty PlayStation controller. Fix this soon before I am outside of my refund window.\n\nEdit: I've spent another 30 minutes trying every janky ass fix you can find online and in the discussions and shockingly none of them work. Seriously, how is this a thing? And no fix yet? CLUELESS FUCKING DEVS MAN, IT'S GETTING SO TIRING!", 'voted_up': False, 'votes_up': 16, 'timestamp_created': 1726793968, 'timestamp_updated': 1726900092, 'title_vector': [-0.030018333345651627, 0.0366851

In [150]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

class ReviewReader:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Ensure the model is passed for generating embeddings

    def read_all_reviews(self):
        """Retrieve all reviews from the index."""
        try:
            response = self.es.search(index=self.index_name, body={"query": {"match_all": {}}})
            return response['hits']['hits']  # returns the list of documents
        except Exception as e:
            print(f"Error retrieving documents: {e}")
            return []

    def read_review_by_appid(self, appid):
        """Retrieve a specific review by appid."""
        try:
            response = self.es.search(index=self.index_name, body={
                "query": {
                    "term": {
                        "appid": appid  # Search by appid
                    }
                }
            })
            return response['hits']['hits']  # returns the list of documents matching the appid
        except Exception as e:
            print(f"Error retrieving document with appid {appid}: {e}")
            return []

    def read_reviews_knn(self, query, title, vector_field="review_vector", num_results=5):
        """Retrieve reviews using KNN search only to test vector querying."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": vector_field,  # Use the specified vector field (e.g., "review_vector")
                "query_vector": knn_vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body={"knn": knn_query})

            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            if not result_docs:
                print("No results found.")
            return result_docs

        except Exception as e:
            print(f"Error executing KNN search: {e}")
            return []
    

    def read_reviews_knn_and_keyword(self, query, title, num_results=5, vector_field="review_vector"):
        """Retrieve reviews using both KNN and keyword search in a single query."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": vector_field,  # Use the specified vector field (e.g., "review_vector")
                "query_vector": knn_vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Define the keyword search query
            keyword_query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question", "review", "title"],  # Relevant fields for keyword search
                            "type": "best_fields",
                            "boost": 0.5
                        }
                    },
                    "filter": {
                        "term": {
                            "title": title  # Filter by title
                        }
                    }
                }
            }

            # Combine the KNN and keyword search queries
            search_query = {
                "knn": knn_query,
                "query": keyword_query,
                "size": num_results,  # Limit the number of results
                "_source": ["review", "appid", "question", "title"]
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            return result_docs

        except Exception as e:
            print(f"Error executing KNN and keyword search: {e}")
            return []

In [151]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

class ReviewReader:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Ensure the model is passed for generating embeddings

    def read_all_reviews(self):
        """Retrieve all reviews from the index."""
        try:
            response = self.es.search(index=self.index_name, body={"query": {"match_all": {}}})
            return response['hits']['hits']  # returns the list of documents
        except Exception as e:
            print(f"Error retrieving documents: {e}")
            return []

    def read_review_by_appid(self, appid):
        """Retrieve a specific review by appid."""
        try:
            response = self.es.search(index=self.index_name, body={
                "query": {
                    "term": {
                        "appid": appid  # Search by appid
                    }
                }
            })
            return response['hits']['hits']  # returns the list of documents matching the appid
        except Exception as e:
            print(f"Error retrieving document with appid {appid}: {e}")
            return []

    def read_reviews_knn(self, query, title, vector_field="review_vector", num_results=5):
        """Retrieve reviews using KNN search only to test vector querying."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": vector_field,  # Use the specified vector field (e.g., "review_vector")
                "query_vector": knn_vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body={"knn": knn_query})

            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            if not result_docs:
                print("No results found.")
            return result_docs

        except Exception as e:
            print(f"Error executing KNN search: {e}")
            return []
    

    def read_reviews_knn_and_keyword(self, field, query, vector, title, num_results=5):
        """Retrieve reviews using both KNN and keyword search in a single query."""
        try:
            # Ensure the model is set
            if self.model is None:
                raise ValueError("Model for embedding generation is not initialized.")

            # Generate the embedding vector from the query
            # knn_vector = self.model.encode(query).tolist()

            # Define the KNN query
            knn_query = {
                "field": field,  # Use the specified vector field (e.g., "review_vector")
                "query_vector": vector,
                "k": num_results,
                "num_candidates": 10000,  # Adjust as necessary
                "boost": 0.5,
                "filter": {
                    "term": {
                        "title": title  # Filter by title
                    }
                }
            }

            # Define the keyword search query
            keyword_query = {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question", "answer", "section"],  # Relevant fields for keyword search
                            "type": "best_fields",
                            "boost": 0.5
                        }
                    },
                    "filter": {
                        "term": {
                            "title": title  # Filter by title
                        }
                    }
                }
            }

            # Combine the KNN and keyword search queries
            search_query = {
                "knn": knn_query,
                "query": keyword_query,
                "size": num_results,  # Limit the number of results
                "_source": ["answer", "section", "question", "title"]
            }

            # Perform the search
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = []

            # Collect results
            for hit in es_results['hits']['hits']:
                result_docs.append(hit['_source'])  # Append only the source of each document

            return result_docs

        except Exception as e:
            print(f"Error executing KNN and keyword search: {e}")
            return []

In [152]:
# # Example Usage

# # Initialize the SentenceTransformer model for embedding generation
# model_name = 'multi-qa-MiniLM-L6-cos-v1'
# model = SentenceTransformer(model_name)

# # Create a reader instance with the model
# reader = ReviewReader(model=model)

# # Example of querying reviews using KNN and keyword search with embeddings
# title = "God of War: Ragnarok"  # Example game title
# # title = "Far Cry 5"  # Example game title
# query = 'Is God of War a game for kids?'
# query = 'Is God of War launching with proper controller support?'

# # Perform the query using different vector fields (e.g., "review_vector")
# for vector_field in ["review_vector", "title_vector", "title_review_vector"]:
#     print(f"\nQuerying with vector field: {vector_field}")
#     results = reader.read_reviews_knn_and_keyword(query=query, title=title, vector_field=vector_field)

#     # Convert the results to a pandas DataFrame if results are not empty
#     if results:
#         df_results = pd.DataFrame(results)
#         pd.reset_option('all')  # Reset pandas options to default (if necessary)
#         print(df_results.head())  # Display the top rows of the DataFrame
#     else:
#         print("No results found.")

In [153]:
# Example of reading all indexed reviews
all_reviews = reader.read_all_reviews()
print("All Indexed Reviews:")
for review in all_reviews:
    print(review['_id'])  # Print the id of each review
    # print(review['_source'])  # Print the source of each review

All Indexed Reviews:
O44MXZIBH8YBboX2vVUX
PI4MXZIBH8YBboX2vVVy


In [154]:
# Example of reading specific review by appid
specific_reviews = reader.read_review_by_appid("2322010")
print("\nSpecific Reviews for appid 2322010  :")
for review in specific_reviews:
    print("-" * 79)
    print(review['_id'])  # Print the source of the specific review
    print(review['_source'])  # Print the source of the specific review


Specific Reviews for appid 2322010  :
-------------------------------------------------------------------------------
PI4MXZIBH8YBboX2vVVy
{'appid': '2322010', 'timestamp_query': 1727643597, 'title': 'God of War: Ragnarok', 'recommendationid': '175341259', 'author.steamid': '76561198053458540', 'author.playtimeforever': None, 'author.playtime_last_two_weeks': 65, 'author.playtime_at_review': 48, 'author.last_played': 1727486181, 'language': 'english', 'review': "This game launching without proper controller support when God of War had it is seriously a shame. Hopefully this is a bug and not some scam to sell you a shitty PlayStation controller. Fix this soon before I am outside of my refund window.\n\nEdit: I've spent another 30 minutes trying every janky ass fix you can find online and in the discussions and shockingly none of them work. Seriously, how is this a thing? And no fix yet? CLUELESS FUCKING DEVS MAN, IT'S GETTING SO TIRING!", 'voted_up': False, 'votes_up': 16, 'timestamp_c

In [155]:
ground_truth_reviews = [
    {
        'title': 'God of War: Ragnarok',
        'review': "Is this game for kids?"
    },
    {
        'title': 'Far Cry 5',
        'review': "Is this game open world with outpost capture mechanics?"
    },
    # Add more ground truth examples as needed
]

In [156]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [157]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [176]:
field='review_vector'
question="Is God of War launching with proper controller support?"
title="God of War: Ragnarok"
title="Far Cry 5"

In [177]:
v_q = model.encode(question)

In [178]:
response = reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, title=title, num_results=5)

In [179]:
response

[{'title': 'Far Cry 5'}]

In [182]:
def question_text_hybrid():
    field='title_review_vector'
    question="Is God of War launching with proper controller support?"
    title="God of War: Ragnarok"
    
    v_q = model.encode(question)

    return reader.read_reviews_knn_and_keyword(field=field, query=question, vector=v_q, title=title, num_results=5)

In [183]:
question_text_hybrid()

[{'title': 'God of War: Ragnarok'}]

In [144]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [145]:
evaluate(ground_truth, question_hybrid)

NameError: name 'ground_truth' is not defined

In [111]:
def review_hybrid_search(title, query, model, reader, vector_field):
    """
    Perform hybrid search using KNN vector search and keyword matching based on review text and title.

    Parameters:
    title (str): Title of the game for filtering.
    query (str): Review text query.
    model (SentenceTransformer): Model for generating vectors.
    reader (ReviewReader): An instance of ReviewReader for querying Elasticsearch.
    vector_field (str): The vector field to be used for KNN search (e.g., 'review_vector').

    Returns:
    list: List of search results (reviews).
    """
    # Generate the vector for the query
    v_query = model.encode(query)
    v_title = model.encode(title)

    # Combine title and query vectors for searching
    combined_vector = (v_query + v_title) / 2  # Averaging the two vectors

    return reader.read_reviews_knn_and_keyword(query=query, title=title, vector_field=vector_field)

In [112]:
def evaluate_reviews(ground_truth, model, reader, vector_field):
    """
    Evaluate the hybrid search results using hit rate and mean reciprocal rank (MRR) based on review and title text.

    Parameters:
    ground_truth (list): A list of dictionaries containing ground truth data with 'review' and 'title'.
    model (SentenceTransformer): Model for generating vectors.
    reader (ReviewReader): An instance of ReviewReader for querying Elasticsearch.
    vector_field (str): The vector field to be used for KNN search (e.g., 'review_vector').

    Returns:
    dict: A dictionary containing the hit rate and MRR scores.
    """
    relevance_total = []

    for review in tqdm(ground_truth):
        # Extract ground truth information
        expected_title = review['title']
        expected_review = review['review']

        # Perform the search
        results = review_hybrid_search(expected_title, expected_review, model, reader, vector_field)

        # Check if any of the retrieved results match the expected review and title
        relevance = [
            result['title'] == expected_title and result['review'] == expected_review 
            for result in results
        ]
        
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [107]:
def hit_rate(relevance_total):
    cnt = sum(1 for line in relevance_total if True in line)
    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:
                total_score += 1 / (rank + 1)
                break
    return total_score / len(relevance_total)


In [114]:
# Example Usage

# Initialize the SentenceTransformer model for embedding generation
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

# Create a reader instance with the model
reader = ReviewReader(model=model)

# Example ground truth data for evaluation
ground_truth_reviews = [
    {
        'title': 'God of War: Ragnarok',
        'review': "Is this game for kids?"
    },
    {
        'title': 'Far Cry 5',
        'review': "Is this game open world with outpost capture mechanics?"
    },
    # Add more ground truth examples as needed
]

# Choose a vector field for the search
vector_field = 'review_vector'  # or 'title_vector', 'title_review_vector'

# Evaluate the search function
results = evaluate_reviews(ground_truth_reviews, model, reader, vector_field)

# Print the evaluation metrics
print(f"Hit Rate: {results['hit_rate']}")
print(f"MRR: {results['mrr']}")


100%|██████████| 2/2 [00:00<00:00, 18.12it/s]

Hit Rate: 0.0
MRR: 0.0


In [109]:
# Example Usage

# Initialize the SentenceTransformer model for embedding generation
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

# Create a reader instance with the model
reader = ReviewReader(model=model)

# Example ground truth data for evaluation
ground_truth_reviews = [
    {
        'title': 'God of War: Ragnarok',
        'review': "This game is not for kids."
    },
    {
        'title': 'Far Cry 5',
        'review': "Is this game open world with outpost capture mechanics?"
    },
    # Add more ground truth examples as needed
]

# Choose a vector field for the search
vector_field = 'review_vector'  # or 'title_vector', 'title_review_vector'

# Evaluate the search function
results = evaluate_reviews(ground_truth_reviews, reader, vector_field)

# Print the evaluation metrics
print(f"Hit Rate: {results['hit_rate']}")
print(f"MRR: {results['mrr']}")


100%|██████████| 2/2 [00:00<00:00, 39.73it/s]

Hit Rate: 0.0
MRR: 0.0


In [82]:
# def question_hybrid(q):
#     question = q['question']
#     course = q['course']

#     v_q = model.encode(question)

#     return elastic_search_hybrid('question_vector', question, v_q, course)

In [32]:
def question_hybrid(q):
    # title = "God of War: Ragnarok"  # Example game title
    # query = 'Is God of War: Ragnarok a game for kids?'
    query=q['question']
    appid=q['appid']
    v_q = model.encode(query)

    return indexer.query_reviews_hybrid(appid, query, v_q, vector_field="review_vector")

In [33]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        print(q)
        
        # doc_id = q['question']
        doc_id = q['appid']
        
        results = search_function(q)
        print(results[0])
        
        relevance = [d['appid'] == doc_id for d in results]
        print(relevance[0])
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [34]:
import json
import os

# Define the data directory path
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the input file path
input_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Check if the file exists before attempting to read
if os.path.exists(input_file):
    # Read the results from the JSON file
    with open(input_file, 'r', encoding='utf-8') as json_file:
        ground_truth = json.load(json_file)  # Load the JSON data into a Python object

    # Now ground_truth contains the data from ground_truth_retrieval.json
    print(f"Data loaded from {input_file}")
    print(f"Number of entries: {len(ground_truth)}")
    # Optionally display the first few records
    print(ground_truth[:5])  # Display the first 5 entries
else:
    print(f"File {input_file} does not exist!")

Data loaded from /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
Number of entries: 105
[{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}, {'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': 'What resolution and settings were used to play God of War: Ragnarok according to the review?'}, {'appid': '2322010', 'question': 'Did the reviewer experience any performance issues such as stutter while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': "Is there any mention of the game's story or gameplay quality in the review?"}]


In [35]:
ground_truth[:1]

[{'appid': '2322010',
  'question': 'How well does God of War: Ragnarok perform on PC at launch?'}]

In [36]:
evaluate(ground_truth, question_hybrid)

  3%|▎         | 3/105 [00:00<00:03, 27.13it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}
{'appid': '2322010', 'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove this VRAM limitation. From personal experience and community feedback, I can confirm that the game runs well on certain integrated graphics processors (such as the Ryzen 3 3200G, Ryzen 5 3400G, and Ryzen 7 5700G), which are generally weaker than many 4GB VRAM GPUs available today. God of War: Ragnarok performs on the base PlayStation 4, which further suggests that the game should be capable of running on 4GB VRAM graphics cards, and the latest 

  6%|▌         | 6/105 [00:00<00:04, 23.03it/s]

{'appid': '2369390', 'review': 'fun game. the story is pretty dumb tbh and there are endless cutscenes. Not sure why such a recent game relies on such janky and clunky storytelling, when games like cyberpunk and baldurs gate 3 respond to player actions so much better.'}
True
{'appid': '2369390', 'question': 'Are there excessive cutscenes in Far Cry 6 that interrupt the gameplay experience?'}
{'appid': '2369390', 'review': "WARNING: DO NOT DOWNLOAD THIS GAME ON AN HDD, THE GAME IS FAR BUGGIER AND YOU WILL EXPERIENCE CRASHES! \n\nFar Cry 6 is both a step forward and a step backward for the Far Cry series, as it is a far cry from the first 5 main titles in its style and offers the most experimentation in the series to date. The gameplay and side content has been improved, with far more variety and complexity added to the regular Far Cry formula, and the open world is absolutely massive, but the series has lost some of its character with the transition to 3rd person cutscenes, and it doesn

  9%|▊         | 9/105 [00:00<00:04, 22.79it/s]

...

 96%|█████████▌| 101/105 [00:03<00:00, 35.74it/s]

{'appid': '1817070', 'review': "Marvel's Spider-Man Remastered may not be as well-rounded an experience as Rocksteady's Arkham series. But when you're swinging through Time Square in glorious sunshine as New York's citizenry gazes at you from below, well, it's a superhero high that even the Caped Crusader would struggle to beat."}
True
{'appid': '1817070', 'question': "What are the main highlights of the gameplay experience in Marvel's Spider-Man Remastered?"}
{'appid': '1817070', 'review': "Marvel's Spider-Man Remastered may not be as well-rounded an experience as Rocksteady's Arkham series. But when you're swinging through Time Square in glorious sunshine as New York's citizenry gazes at you from below, well, it's a superhero high that even the Caped Crusader would struggle to beat."}
True
{'appid': '1817070', 'question': "Is Marvel's Spider-Man Remastered available for purchase on Steam?"}
{'appid': '1817070', 'review': "After completing Marvel’s Spider-Man Remastered, finishing 100

100%|██████████| 105/105 [00:03<00:00, 32.08it/s]

{'appid': '794540', 'review': "Neo Cab's broader social critiques really hit the mark. We play Lina, a cab driver trying to make it day to day in a not-to-distant-future where surveillance, technology and capitalism have all but removed every last freedom for the individual.\r\nThe gameplay is relatively simple but with only a 4 hour runtime, it doesn't need to be anything more. However, the broader personal narrative for the protagonist and the limited options we have at expressing our own agency within the story left me frustrated. Maybe it was my decisions throughout the game that caused my Lina to become everyone's doormat, but the story resolved in a way where I never had the chance to stand up for myself, no matter how much I wanted to, and that left a bitter taste in my mouth.\r\nI would certainly recommend Neo Cab and I would be keen to hear about other's experiences with the game."}
True
{'appid': '794540', 'question': 'Is there a built-in system for saving progress in Neo Cab

{'hit_rate': 1.0, 'mrr': 1.0}

In [37]:
# {'appid': '1817070', 'question': "Is Marvel's Spider-Man Remastered a well-optimized game upon release?"}
# {'appid': '1817070', 'review': "This game is absolutely amazing, its extremely well optimized, great graphics, i dont get tired at looking the beautiful different suits, one of the best stories i've seen in videogames. If you like Spider-Man buy this game. If you liked Spider-Man just a little bit as a kid, like me, buy this game. Even if you don't care much about Spider-Man but like a well-written, solid game, buy this game."}
# True

In [38]:
query="Is Marvel's Spider-Man Remastered a well-optimized game upon release?"

In [39]:
appid="1817070"

In [40]:
v_q = model.encode(query)

In [41]:
results = indexer.query_reviews_hybrid(appid, query, v_q, vector_field="title_review_vector")

In [42]:
results

[{'appid': '1817070',
  'review': "Marvel's Spider-Man Remastered may not be as well-rounded an experience as Rocksteady's Arkham series. But when you're swinging through Time Square in glorious sunshine as New York's citizenry gazes at you from below, well, it's a superhero high that even the Caped Crusader would struggle to beat."},
 {'appid': '1817070',
  'review': "This game is absolutely amazing, its extremely well optimized, great graphics, i dont get tired at looking the beautiful different suits, one of the best stories i've seen in videogames. If you like Spider-Man buy this game. If you liked Spider-Man just a little bit as a kid, like me, buy this game. Even if you don't care much about Spider-Man but like a well-written, solid game, buy this game."}]

In [43]:
import json
import os

# Define the data directory path
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the input file path
input_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Check if the file exists before attempting to read
if os.path.exists(input_file):
    # Read the results from the JSON file
    with open(input_file, 'r', encoding='utf-8') as json_file:
        ground_truth = json.load(json_file)  # Load the JSON data into a Python object

    # Now ground_truth contains the data from ground_truth_retrieval.json
    print(f"Data loaded from {input_file}")
    print(f"Number of entries: {len(ground_truth)}")
    # Optionally display the first few records
    print(ground_truth[:5])  # Display the first 5 entries
else:
    print(f"File {input_file} does not exist!")

Data loaded from /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
Number of entries: 105
[{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}, {'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': 'What resolution and settings were used to play God of War: Ragnarok according to the review?'}, {'appid': '2322010', 'question': 'Did the reviewer experience any performance issues such as stutter while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': "Is there any mention of the game's story or gameplay quality in the review?"}]


In [44]:
ground_truth[:2]

[{'appid': '2322010',
  'question': 'How well does God of War: Ragnarok perform on PC at launch?'},
 {'appid': '2322010',
  'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}]

In [45]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [46]:
def build_prompt(query, search_results):
    answer_llm = ""

    if 'entry_template' in globals():  # Check if entry_template exists in global scope
        for doc in search_results:
            answer_llm += entry_template.format(**doc) + "\n\n"
    else:
        for doc in search_results:
            # Fallback formatting if entry_template is missing
            answer_llm += str(doc) + "\n\n"

    prompt = prompt_template.format(question=query, answer_llm=answer_llm).strip()
    return prompt

In [47]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [48]:
def rag(query, model='gpt-4o-mini'):
    # search_results = search(query)
    search_results = indexer.query_reviews_hybrid(appid, query, v_q, vector_field=vector_field)
    print(search_results)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    # print(answer)
    return answer

In [49]:
prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [50]:
# evaluations = []

# for record in tqdm(ground_truth):
#     question = record['question']
#     print(question)
#     answer_llm = rag(question) 

#     prompt = prompt_template.format(
#         question=question,
#         answer_llm=answer_llm
#     )

#     evaluation = llm(prompt)
#     evaluation = json.loads(evaluation)

#     evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/105 [00:00<?, ?it/s]

How well does God of War: Ragnarok perform on PC at launch?
[{'appid': '1817070', 'review': 'I was a little skeptical at first on how the game would translate to PC controls with it being originally made with a controller in mind. While it is a bit of a learning curve it was pretty easy to get used to the controls. The game itself is still a damn good game with better graphics! Highly recommend this game for any fans of Spider-man.'}, {'appid': '1817070', 'review': "I've only played this game (at this time) for about an hour, but I can confidently say that I will be returning to play this game. The gameplay is just unmatched. The only problem of this game is that you require a decent pc to run this game. Other than that, the mechanics are very well made. Swinging around at night while listening to music is another kind of vibe that can only be achieved by playing this masterpiece."}]


  1%|          | 1/105 [00:03<05:43,  3.30s/it]

What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?
[{'appid': '1817070', 'review': "I've only played this game (at this time) for about an hour, but I can confidently say that I will be returning to play this game. The gameplay is just unmatched. The only problem of this game is that you require a decent pc to run this game. Other than that, the mechanics are very well made. Swinging around at night while listening to music is another kind of vibe that can only be achieved by playing this masterpiece."}, {'appid': '1817070', 'review': "Fantastic game.\r\n\r\nThe highlight is truly the traversal system, the way Spiderman moves is simply amazing.\r\nHighly addictive, I rarely used the fast forward because it was so enticing to travel through the city.\r\n\r\nThe combat and mission are also diverse enough not to become stale and repetitive over time, at first I wasn't too fond of the Gadgets but they are very interesting and amusing to use.\r\n\r\nStor

  2%|▏         | 2/105 [00:05<04:41,  2.73s/it]

...

 98%|█████████▊| 103/105 [05:52<00:08,  4.23s/it]

What themes are explored in Neo Cab through its narrative and character interactions?
[{'appid': '1817070', 'review': 'Overall: 8.75/10\n\nStory:\n\nSpider-Man Remastered is an open world action-adventure game that delivers an engaging, character-driven story that expertly blends superhero action with personal drama. Set eight years into Peter Parker\'s career as Spider-Man, the narrative masterfully intertwines Peter’s personal life with his responsibilities as Spider-Man, while acknowledging the delicate balance between his life as a superhero and his personal life and relationships, delivering a story rich with emotional depth.\nThe game portrays Peter as an experienced superhero juggling his role as a protector of New York with his struggles in everyday life, creating a compelling dual narrative. His relationships with many recognizable key characters are deeply explored, adding layers of personal stakes to the story.\nThe game’s villain lineup is also very well-developed, offering

 99%|█████████▉| 104/105 [05:55<00:03,  3.78s/it]

Is there a built-in system for saving progress in Neo Cab, and how does it work?
[{'appid': '1817070', 'review': "Spider-Man Remastered really takes an already amazing game and just makes it so much better. Visual improvements in this are stunning, and swinging through New York feels smoother than ever. Even the faster load times and updated character models give it a different feel, but it still retains all the fun of the original. It still retains a story that is just as gripping, and the combat is super fluid, satisfying. Honestly, a replay or jumping in for the first time, it is one of the best superhero games out there.\r\n\r\nJust incredible visual improvements, smoother mechanics, and faster load times take an already great game in Spider-Man Remastered and elevate the experience to a whole new level. The story remains compelling and emotionally engaging, while the combat and traversal mechanics continue to shine, offering a fluid and dynamic gameplay experience. However, the op

100%|██████████| 105/105 [05:58<00:00,  3.42s/it]


In [51]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['appid'] = df_eval.record.apply(lambda d: d['appid'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['explanation'])

del df_eval['record']
del df_eval['evaluation']

In [52]:
df_eval.relevance.value_counts()

relevance
NON_RELEVANT       78
PARTLY_RELEVANT    20
RELEVANT            7
Name: count, dtype: int64

In [53]:
df_eval.relevance.value_counts(normalize=True)

relevance
NON_RELEVANT       0.742857
PARTLY_RELEVANT    0.190476
RELEVANT           0.066667
Name: proportion, dtype: float64

In [54]:
df_eval[df_eval["relevance"]=="RELEVANT"]

,answer,appid,question,relevance,explanation
4,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,Is there any mention of the game's story or ga...,RELEVANT,The generated answer clearly identifies that b...
17,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",582160,"Does the game require grinding to progress, or...",RELEVANT,The generated answer directly addresses the qu...
19,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",582160,How does the game perform in terms of graphics...,RELEVANT,The generated answer effectively addresses the...
48,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",1680880,Does the review suggest that the game's writin...,RELEVANT,The generated answer explicitly states that th...
74,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1086940,What are the reviews saying about the game's p...,RELEVANT,The generated answer specifically addresses th...
95,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1817070,Is Marvel's Spider-Man Remastered a well-optim...,RELEVANT,The generated answer directly addresses the qu...
98,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1817070,What are the main highlights of the gameplay e...,RELEVANT,The generated answer effectively outlines the ...


In [55]:
df_eval[df_eval["relevance"]=="NON_RELEVANT"]

,answer,appid,question,relevance,explanation
1,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What kind of PC specifications did the reviewe...,NON_RELEVANT,The generated answer does not address the ques...
2,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What resolution and settings were used to play...,NON_RELEVANT,The generated answer fails to address the spec...
3,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Did the reviewer experience any performance is...,NON_RELEVANT,The generated answer fails to address the spec...
5,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,"Is the storytelling in Far Cry 6 engaging, or ...",NON_RELEVANT,The generated answer does not address the ques...
6,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,Are there excessive cutscenes in Far Cry 6 tha...,NON_RELEVANT,The generated answer focuses completely on Spi...
...,...,...,...,...,...
99,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",1817070,Is Marvel's Spider-Man Remastered available fo...,NON_RELEVANT,The generated answer fails to address the spec...
101,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Does Neo Cab deliver on the promises made by i...,NON_RELEVANT,The generated answer does not address the ques...
102,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,How does the dialogue in Neo Cab contribute to...,NON_RELEVANT,The generated answer does not address the ques...
103,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,What themes are explored in Neo Cab through it...,NON_RELEVANT,The generated answer focuses entirely on 'Spid...


In [56]:
df_eval[df_eval["appid"]=="2322010"]

,answer,appid,question,relevance,explanation
0,"{\n ""relevance"": ""PARTLY_RELEVANT"",\n ""expla...",2322010,How well does God of War: Ragnarok perform on ...,PARTLY_RELEVANT,The generated answer touches on aspects of gam...
1,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What kind of PC specifications did the reviewe...,NON_RELEVANT,The generated answer does not address the ques...
2,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What resolution and settings were used to play...,NON_RELEVANT,The generated answer fails to address the spec...
3,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Did the reviewer experience any performance is...,NON_RELEVANT,The generated answer fails to address the spec...
4,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,Is there any mention of the game's story or ga...,RELEVANT,The generated answer clearly identifies that b...


In [57]:
import json
import os
import pandas as pd

# Sample DataFrame for demonstration
# df_eval = pd.DataFrame({'recommendationid': [1, 2], 'question': ['What is the best game?', 'What is the worst game?']})

data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_evaluation.json")
6
# Convert DataFrame to a list of dictionaries including index
df_eval_list = df_eval.reset_index().to_dict(orient='records')

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(df_eval_list, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")

Data saved to /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_evaluation.json


In [58]:
reviews[0]

{'appid': '812140',
 'timestamp_query': 1727643597,
 'title': "Assassin's Creed Odyssey",
 'recommendationid': '174160338',
 'author.steamid': '76561198420943538',
 'author.playtimeforever': 0,
 'author.playtime_last_two_weeks': 0,
 'author.playtime_at_review': 5599,
 'author.last_played': 1696845057,
 'language': 'english',
 'review': "---{ Graphics }---\n✅ You forget what reality is\n☐ Beautiful\n☐ Good\n☐ Decent\n☐ Bad\n☐ You will get eye cancer\n☐ Get a pepper spray for your eye instead\n\n---{ Gameplay }---\n☐ Won’t ever touch any other game anymore\n✅ Very good\n☐ Good\n☐ It's just gameplay\n☐ Mehh\n☐ Watch paint dry instead\n☐ Tic Tac toe is better\n\n---{ Audio }---\n☐ Eargasm\n✅ Very good\n☐ Good\n☐ Not too bad\n☐ Bad\n☐ I'm now deaf\n\n---{ Audience }---\n☐ Kids\n✅Teens\n✅ Adults\n☐ Grandma\n\n---{ PC Requirements }---\n☐ Check if you can run paint\n☐ Potato\n☐ Decent\n✅ Fast\n☐ Rich boi\n☐ Ask NASA if they have a spare computer\n☐ Search the galaxy for dark matter fuel to ru

In [59]:
reviews[-1]

{'appid': '2239550',
 'timestamp_query': 1727643597,
 'title': 'Watch Dogs: Legion',
 'recommendationid': '174862159',
 'author.steamid': '76561199371555316',
 'author.playtimeforever': 0,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 109,
 'author.last_played': 1726993288,
 'language': 'english',
 'review': 'meh',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726233314,
 'timestamp_updated': 1726233314,
 'title_vector': [-0.031208040192723274,
  -0.04140456020832062,
  0.007626562379300594,
  -0.05640276148915291,
  0.017036626115441322,
  0.07028614729642868,
  -0.02655501663684845,
  -0.03512350469827652,
  0.05046771839261055,
  -0.004683449864387512,
  0.0064065903425216675,
  -0.034316763281822205,
  0.06852371990680695,
  -0.0006094787968322635,
  0.007349507883191109,
  -0.016963135451078415,
  0.0868942141532898,
  0.009798023849725723,
  0.03648601472377777,
  -0.01720307394862175,
  -0.06752648204565048,
  -0.03290846198797226,
  0.010983460

In [60]:
import json
from tqdm import tqdm

evaluations = []

for record in tqdm(ground_truth):
    question = record['question']
    print(question)
    
    # Get LLM answer
    answer_llm = rag(question) 

    # Format the prompt with the question and LLM answer
    prompt = prompt_template.format(
        question=question,
        answer_llm=answer_llm
    )

    # Get evaluation from LLM
    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    # Append tuple with the question, record, LLM answer, and evaluation
    evaluations.append((question, record, answer_llm, evaluation))


  0%|          | 0/105 [00:00<?, ?it/s]

How well does God of War: Ragnarok perform on PC at launch?
[{'appid': '1817070', 'review': 'I was a little skeptical at first on how the game would translate to PC controls with it being originally made with a controller in mind. While it is a bit of a learning curve it was pretty easy to get used to the controls. The game itself is still a damn good game with better graphics! Highly recommend this game for any fans of Spider-man.'}, {'appid': '1817070', 'review': "I've only played this game (at this time) for about an hour, but I can confidently say that I will be returning to play this game. The gameplay is just unmatched. The only problem of this game is that you require a decent pc to run this game. Other than that, the mechanics are very well made. Swinging around at night while listening to music is another kind of vibe that can only be achieved by playing this masterpiece."}]


  1%|          | 1/105 [00:07<12:18,  7.10s/it]

What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?
[{'appid': '1817070', 'review': "I've only played this game (at this time) for about an hour, but I can confidently say that I will be returning to play this game. The gameplay is just unmatched. The only problem of this game is that you require a decent pc to run this game. Other than that, the mechanics are very well made. Swinging around at night while listening to music is another kind of vibe that can only be achieved by playing this masterpiece."}, {'appid': '1817070', 'review': "Fantastic game.\r\n\r\nThe highlight is truly the traversal system, the way Spiderman moves is simply amazing.\r\nHighly addictive, I rarely used the fast forward because it was so enticing to travel through the city.\r\n\r\nThe combat and mission are also diverse enough not to become stale and repetitive over time, at first I wasn't too fond of the Gadgets but they are very interesting and amusing to use.\r\n\r\nStor

  2%|▏         | 2/105 [00:10<08:44,  5.09s/it]

...

 98%|█████████▊| 103/105 [05:40<00:06,  3.29s/it]

What themes are explored in Neo Cab through its narrative and character interactions?
[{'appid': '1817070', 'review': 'Overall: 8.75/10\n\nStory:\n\nSpider-Man Remastered is an open world action-adventure game that delivers an engaging, character-driven story that expertly blends superhero action with personal drama. Set eight years into Peter Parker\'s career as Spider-Man, the narrative masterfully intertwines Peter’s personal life with his responsibilities as Spider-Man, while acknowledging the delicate balance between his life as a superhero and his personal life and relationships, delivering a story rich with emotional depth.\nThe game portrays Peter as an experienced superhero juggling his role as a protector of New York with his struggles in everyday life, creating a compelling dual narrative. His relationships with many recognizable key characters are deeply explored, adding layers of personal stakes to the story.\nThe game’s villain lineup is also very well-developed, offering

 99%|█████████▉| 104/105 [05:43<00:03,  3.32s/it]

Is there a built-in system for saving progress in Neo Cab, and how does it work?
[{'appid': '1817070', 'review': "Spider-Man Remastered really takes an already amazing game and just makes it so much better. Visual improvements in this are stunning, and swinging through New York feels smoother than ever. Even the faster load times and updated character models give it a different feel, but it still retains all the fun of the original. It still retains a story that is just as gripping, and the combat is super fluid, satisfying. Honestly, a replay or jumping in for the first time, it is one of the best superhero games out there.\r\n\r\nJust incredible visual improvements, smoother mechanics, and faster load times take an already great game in Spider-Man Remastered and elevate the experience to a whole new level. The story remains compelling and emotionally engaging, while the combat and traversal mechanics continue to shine, offering a fluid and dynamic gameplay experience. However, the op

100%|██████████| 105/105 [05:46<00:00,  3.30s/it]


In [62]:
df_eval = pd.DataFrame(evaluations, columns=['question', 'record', 'answer', 'evaluation'])

df_eval['appid'] = df_eval.record.apply(lambda d: d['appid'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['explanation'])

del df_eval['record']
del df_eval['evaluation']

In [63]:
df_eval.relevance.value_counts()

relevance
NON_RELEVANT       78
PARTLY_RELEVANT    18
RELEVANT            9
Name: count, dtype: int64

In [64]:
df_eval.relevance.value_counts(normalize=True)

relevance
NON_RELEVANT       0.742857
PARTLY_RELEVANT    0.171429
RELEVANT           0.085714
Name: proportion, dtype: float64

In [65]:
df_eval[df_eval["relevance"]=="RELEVANT"]

,question,answer,appid,relevance,explanation
4,Is there any mention of the game's story or ga...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,RELEVANT,The generated answer directly addresses the qu...
17,"Does the game require grinding to progress, or...","{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",582160,RELEVANT,The generated answer directly addresses the qu...
19,How does the game perform in terms of graphics...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",582160,RELEVANT,The generated answer directly addresses the qu...
48,Does the review suggest that the game's writin...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",1680880,RELEVANT,The generated answer directly addresses the qu...
49,What reasons did the reviewer give for feeling...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1680880,RELEVANT,The generated answer addresses the reviewer's ...
74,What are the reviews saying about the game's p...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1086940,RELEVANT,The generated answer comprehensively addresses...
95,Is Marvel's Spider-Man Remastered a well-optim...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1817070,RELEVANT,The generated answer effectively addresses the...
96,How does Marvel's Spider-Man Remastered compar...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1817070,RELEVANT,The generated answer directly addresses the co...
98,What are the main highlights of the gameplay e...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1817070,RELEVANT,The generated answer thoroughly addresses the ...


In [66]:
df_eval[df_eval["relevance"]=="NON_RELEVANT"]

,question,answer,appid,relevance,explanation
1,What kind of PC specifications did the reviewe...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer completely fails to addre...
2,What resolution and settings were used to play...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer does not address the spec...
3,Did the reviewer experience any performance is...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer fails to address the ques...
5,"Is the storytelling in Far Cry 6 engaging, or ...","{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,NON_RELEVANT,The generated answer focuses entirely on Spide...
6,Are there excessive cutscenes in Far Cry 6 tha...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,NON_RELEVANT,The generated answer completely ignores the qu...
...,...,...,...,...,...
100,Is Neo Cab a fully optimized game that functio...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,NON_RELEVANT,The generated answer does not address the spec...
101,Does Neo Cab deliver on the promises made by i...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,NON_RELEVANT,The generated answer does not mention Neo Cab ...
102,How does the dialogue in Neo Cab contribute to...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,NON_RELEVANT,The generated answer focuses entirely on 'Spid...
103,What themes are explored in Neo Cab through it...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,NON_RELEVANT,The generated answer completely deviates from ...


In [67]:
df_eval[df_eval["appid"]=="2322010"]

,question,answer,appid,relevance,explanation
0,How well does God of War: Ragnarok perform on ...,"{\n ""relevance"": ""PARTLY_RELEVANT"",\n ""expla...",2322010,PARTLY_RELEVANT,The generated answer touches on gameplay aspec...
1,What kind of PC specifications did the reviewe...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer completely fails to addre...
2,What resolution and settings were used to play...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer does not address the spec...
3,Did the reviewer experience any performance is...,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,NON_RELEVANT,The generated answer fails to address the ques...
4,Is there any mention of the game's story or ga...,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,RELEVANT,The generated answer directly addresses the qu...


In [69]:
df_eval.to_json('df_eval.json', orient='records', indent=4)